# Requerimientos

In [59]:
import os
from tkinter import Tk, filedialog
from numpy import repeat
import cv2
from win32con import (SW_SHOW, SW_RESTORE)
import win32gui
import win32ui
import shutil
import zipfile
import datetime

Si hay problemas al ejecutarse desde terminal, utilizar el comando:
pip install pywin32 --upgrade

La carpeta que se seleccionará debe de incluir las imagenes que se reetiquetarán ya que es muy probable que el modelo se haya equivocado en estos casos. Una consideración importante es que el nombre de la carpeta no debe incluir "_" pues el programa toma en consideración todo el path

Abriendo carpeta que contiene las imagenes desde ventana Windows

In [60]:
root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
carpeta = filedialog.askdirectory() # Returns opened path as str


In [61]:
# get base name of carpeta
base_name_carpeta = os.path.basename(carpeta)

#base_name_carpeta contains "_"
if "_" in base_name_carpeta:
    d=win32ui.MessageBox("El nombre de la carpeta contiene ( _ ) desea cambiarlo? ", "Error", 1) #1 Aceptar 2 Cancelar
    if d==1:
        #rename carpeta path
        os.rename(carpeta, carpeta.replace("_", ""))
        carpeta=carpeta.replace("_", "")
        #base_name_carpeta = base_name_carpeta.replace("_", "")
    elif d==2:
        exit()
    


In [62]:
os.chdir(carpeta) # Changes directory to opened path.
files = os.listdir()

# if the folder is empty, the program will exit
if len(files) == 0:
    win32ui.MessageBox("La carpeta esta vacia", "Error",0)
    exit()

if not all(file.endswith('.jpg') for file in files):
    win32ui.MessageBox("La carpeta solo debe de contener archivos JPG", "Error",0)
    exit()    
    

In [63]:
# get path of all files
paths = [os.path.join(carpeta, file) for file in files]

Aquí vamos a filtrar el hecho de que en la carpeta seleccionada existan errores en la etiqueta, si es así, el programa continua, caso contrario termina.

In [64]:
# this function ditects a string inside a string
def detect_string(string, substring):
    return substring in string

error_paths = [path for path in paths if detect_string(path, "Error")]    

# if error_paths is not empty, the program will exit
if len(error_paths) == 0:
    win32ui.MessageBox("La carpeta no contiene archivos con error :)", "Mensaje",0)
    exit()

In [65]:
ec = "S:/OMNI/ErroresCorregidos"
# ec = "C:/Users/fcolin/Documents"


# if exists ec folder, the program continues

if not os.path.exists(ec):
    win32ui.MessageBox("Es necesario tener acceso a la carpeta S:\OMNI", "Error",0)
    exit()
    

if os.path.exists(ec):
    # print("La carpeta de errores corregidos existe")
    pass
else:
    os.mkdir(ec)

In [66]:
inicio=win32ui.MessageBox("Para reetiquetar las imagenes de acuerdo a la posición del item, utiliza las teclas: \n\n W: Frontal \n S: Trasera \n A: Izquierda \n D: Derecha", "Instrucciones Generales",1)
if inicio==2:
    exit()

consideraciones = win32ui.MessageBox("Asegurate de tener acceso a la carpeta compartida en donde se guardarán los cambios realizados\n\nLa carpeta debe contener archivos JPG ya etiquetados\n\nEl nombre de la carpeta no debe contener ( _ )\n\nUna ventana emergente mostrará la vista elegida, si desea cambiarla presione la opción Cancelar y escoja nuevamente entre W,A,S o D\n\nUna vez inciado el programa podrá salir hasta que se hayan reetiquetado todas las imagenes de la carpeta", "Consideraciones",1)
if consideraciones==2:
    exit()

In [67]:
#Functions --------------------------------------------------------------------------------------------------
# this function is used to create a subset from a string
def get_subset(string, start, end):
    return string[start:end]    

# this function detects the position of a  character in a string
def detect_char(string, char):
    return [i for i, letter in enumerate(string) if letter == char]  

# With these functions we can activate the window and bring it to the front.
def get_windows_placement(window_id):
    return win32gui.GetWindowPlacement(window_id)[1]

def set_active_window(window_id):
    if get_windows_placement(window_id) == 2:
        win32gui.ShowWindow(window_id, SW_RESTORE)
    else:
        win32gui.ShowWindow(window_id, SW_SHOW)
    win32gui.SetForegroundWindow(window_id)
    win32gui.SetActiveWindow(window_id)


In [68]:
# now we use the previous function to get all '_'
error_paths_positions = [detect_char(path, '_') for path in error_paths]
#select the first position of each path
error_paths_positions_1 = [path[1] for path in error_paths_positions]
error_paths_positions_1 = [int(path) + 1 for path in error_paths_positions_1]

error_paths_positions_2 = [path[2] for path in error_paths_positions]

# get the subset of the paths
error_paths_subset = [get_subset(path, error_paths_positions_1[i], error_paths_positions_2[i]) for i, path in enumerate(error_paths)]

# unique values of error_paths_subset
skue = list(set(error_paths_subset))


Se substrae el SKU con etiqueta erronea, a partir de ahi se subrtaen toooodos los productos con ese SKU


In [69]:
productos_error = []
for path in paths:
    for j in range(len(skue)):
        if detect_string(path, skue[j]):
            # save the path in the new variable
            productos_error.append(path)
            
carpeta = os.getcwd()

# if not exists the folder 'error', the program will create it
if not os.path.exists(carpeta + '\\Errores'):
    os.mkdir("Errores")

for path in productos_error:
    name = os.path.basename(path)
    os.rename(path, carpeta+"/Errores/" + name) 


In [70]:
# files in the folder 'error'
error_files = os.listdir(carpeta + '\\Errores')

Seleccionando solo las imagenes con medida 515 para mostrarse en pantalla

In [71]:
# change directory to 'error'
os.chdir(carpeta + '\\Errores')

In [72]:
# get all paths in directory
files = os.listdir()
# get all paths of files
paths = [os.path.join(carpeta + '\\Errores', file) for file in files]

A todo lo que no contenga _Error_ en su etiqueta se le colocará -new

In [73]:
error_paths = [path for path in paths if detect_string(path, "_Error_")]  

no_error_paths = [path for path in paths if not detect_string(path, "_Error_")]  

for i in range(len(no_error_paths)):
    imagen = no_error_paths[i]
    tipo = (get_subset(imagen, detect_char(imagen, '_')[0]+1, detect_char(imagen, '_')[1]+1) )
    
    tipo2 = tipo.replace("_", "-new_")
    os.rename(imagen, imagen.replace(tipo, tipo2))
        

In [74]:
    principales = [error_paths for error_paths in error_paths if "515Wx515" in error_paths]

# Open cv2

Para que el uso del programa sea mas fluido en el archivo Ejecutable.py se agregan las dos ultimas funciones las cuales colocan sobre el resto de ventanas la imagen a reetiquetar y al mismo tiempo se activa dicha ventana.

* Mostrando las imagenes y utilizando las teclas (w,a,s,d) para etiquetarlas
* Se muestra la imagen y posteriormente la etiqueta que se le asignó mediante el teclado para que el usuario corrobore su selección
* Incluye un bucle que se rompe hasta que se selecciona la opción aceptar (decision final)
* Se renombran con el formato "-new" para evitar repeticiones con los archivos ya existentes

In [75]:
def seleccion(imagen): 
    img = cv2.imread(imagen)
    cv2.imshow('Imagen',img) #
    cv2.setWindowProperty('Imagen', cv2.WND_PROP_TOPMOST, 1)
    window_id = win32gui.GetActiveWindow()#
    set_active_window(window_id)#
    #detecting which key is pressed 
    tecla=cv2.waitKey(0) # waits until a key is pressed
    
    # Pressing only keys predefined
    while tecla != ord('a') and tecla != ord('d') and tecla != ord('w') and tecla != ord('s'):
        win32ui.MessageBox("Recuerda que para etiquetar las fotos debes de presionar las teclas \t W A S D", "Cuidado", 0)
        img = cv2.imread(imagen)
        cv2.imshow('Imagen',img) #
        cv2.setWindowProperty('Imagen', cv2.WND_PROP_TOPMOST, 1)
        window_id = win32gui.GetActiveWindow()#
        set_active_window(window_id)#
        tecla=cv2.waitKey(0) 
        cv2.destroyAllWindows() 

    # Conditional statement to detect which key is pressed
    if tecla == ord('a'):
        new_label="_Izquierda-new_"
    elif tecla == ord('d'):
        new_label="_Derecha-new_" 
    elif tecla == ord('w'):
        new_label="_Frontal-new_"
    elif tecla == ord('s'):
        new_label="_Trasera-new_"
    
    return new_label
      

+ Si se denominan dos imagenes como "frontales" por ejemplo, la primera conserva su nombre y la segunda cambia su no_vista 
+ Solo se aceptan dos imagenes del mismo tipo, (p ejem 2 izquierdas)


In [76]:
for i in range(len(principales)):
    imagen = principales[i]

    new_label=seleccion(imagen)
    
    #Now, if the user wants to change his/her selection, he/ she can select the option "Cancel"
    continuar = win32ui.MessageBox(new_label, "Seleccion", 1) #?

    while continuar == 2:
        new_label=seleccion(imagen)
        continuar = win32ui.MessageBox(new_label, "Seleccion", 1)

    cv2.destroyAllWindows() # destroys the window showing image  
      
    # Elements of the path 
    tipo=get_subset( imagen, detect_char(imagen, '_')[0], detect_char(imagen, '_')[1]+1) #_Derecha_
    sku =get_subset( imagen, detect_char(imagen, '_')[1]+1, detect_char(imagen, '_')[2]) #1700490094
    no_vista=get_subset(imagen, detect_char(imagen, '_')[2], detect_char(imagen, '_')[2]+3) #_1.

    item = [path for path in paths if sku in path] 
    vista=[item for item in item if tipo in item] 

    # Selecting all paths with sku in them and with determinated vista
    item_vista=[vista for vista in vista if no_vista in vista] 
    
    # if imagen2.replace(tipo, new_label) does not exist, rename imagen2 to imagen2.replace(tipo, new_label)
    for j in range(len(item_vista)) :
        imagen2 = item_vista[j]
        
        if not os.path.exists(imagen2.replace(tipo, new_label)):
            os.rename(imagen2, imagen2.replace(tipo, new_label))
            
            
        else:
            if not os.path.exists(imagen2.replace(no_vista, "_1.")):
                imagen3 = imagen2.replace(no_vista, "_1.")
                imagen3 = imagen3.replace(tipo, new_label)
                os.rename(imagen2, imagen3)
                
            else:
                if not os.path.exists(imagen2.replace(no_vista, "_2.")):
                    imagen3 = imagen2.replace(no_vista, "_2.")
                    imagen3 = imagen3.replace(tipo, new_label)
                    os.rename(imagen2, imagen3)
                    

Se revisa que todos los nombres se actualizaron, si no es así, es porque se asigaron mas formatos de los que son permitidos (por ejemplo se seleccionaron 3 imagenes como frontales, sin embargo solo se aceptan 2). Si es el caso, se mueve hacia una direccion que vivirá al mismo nivel que la carpeta padre.

Es importante guardar los cambios que se generen para poder retroalimentar el modelo, por esta razón se tambien se guardan las imagenes reetiquetadas en TomCargasMasivas (carpeta compartida a la que se tiene acceso)

In [77]:
files= os.listdir()

error_files
import pandas as pd
# create a table with files and error files as columns
table = pd.DataFrame(columns=['Files', 'Error Files'])
table['Files'] = files
table['Error Files'] = error_files


# if table has same value in both columns and in the same row, delete that row
table = table.drop_duplicates(subset=['Files', 'Error Files'], keep='first')


In [78]:

files=os.listdir()
for i in range(len(files)):
    if '-' not in files[i]:
        base_name=carpeta[:-len(os.path.basename(carpeta))]
        if not os.path.exists(base_name+"/Repeticiones"):
            os.mkdir(base_name+"/Repeticiones")
        os.rename(files[i], base_name+"/Repeticiones/"+files[i])
        
        
files= os.listdir()
for i in range(len(files)):
    os.rename(files[i],files[i].replace('-new', ''))

# if there exists a folder called "Repeticiones" show a message
if os.path.exists(base_name_carpeta+"Repeticiones"):
    win32ui.MessageBox("Existen Archivos con mas de 2 vistas, revisar carpeta repeticiones", "Error", 0)
    

In [24]:
table = pd.DataFrame(columns=['Files', 'Error Files'])

['1200Wx1200H_Trasera-new_2003674022_1.jpg',
 '300Wx300H_Trasera-new_2003674022_1.jpg',
 '30Wx30H_Trasera-new_2003674022_1.jpg',
 '400Wx600H_Trasera-new_2003674022_1.jpg',
 '40Wx40H_Trasera-new_2003674022_1.jpg',
 '515Wx515H_Trasera-new_2003674022_1.jpg',
 '65Wx65H_Trasera-new_2003674022_1.jpg',
 '96Wx96H_Trasera-new_2003674022_1.jpg']

Creando reporte de cambios

In [81]:
day = datetime.datetime.now().strftime("%d-%m-%Y")  
hour = datetime.datetime.now().strftime("%H-%M-%S")

if os.path.exists("S:/OMNI"):
    if not os.path.exists(ec):
        os.mkdir(ec)

    if not os.path.exists(ec+"\\Transformaciones("+day+"_"+hour+").csv"):    
        table.to_csv(ec+"\\Transformaciones("+day+"_"+hour+").csv", index=False)
else:
     table.to_csv(carpeta+"\\ENVIAR-A-RESPONSABLE-Transformaciones("+day+"_"+hour+").csv", index=False)       


In [26]:
# get all paths in directory
files = os.listdir()
# get all paths of files    
paths = [os.path.join(carpeta + '\\Errores', file) for file in files]


# move all files to carpeta
for i in range(len(paths)):
    os.rename(paths[i], carpeta + "\\" + files[i])


Si carpeta de Errores se encuentra vacia eliminarla 

In [27]:
os.chdir(carpeta)

# if Errores folder empty, delete it
if os.listdir(carpeta + "\\Errores") == []:    
    os.rmdir(carpeta + "\\Errores")
else:
    win32ui.MessageBox("Hubo un error, revisar carpeta", "Error", 0)

In [28]:
Fuera = carpeta.replace(base_name_carpeta, "")
os.chdir(Fuera)

EC = carpeta.replace(base_name_carpeta, "ErroresCorregidos")
# changing name of carpeta to carpeta_new
os.rename(carpeta, EC)


In [29]:
resultados = carpeta.replace(base_name_carpeta, "Resultado-ErroresCorregidos")

# create a folder called Resultado-ErroresCorregidos
if not os.path.exists(resultados):
    os.mkdir(resultados)

In [30]:
# get the user name 
user = os.getenv('USERNAME')

Aplicacion = "C:/Users/"+ user +"/Downloads/Aplicacion/Codigo-IMPEX/R-4.2.1/bin"

#change directory to Aplicacion
os.chdir(Aplicacion)

os.system("Rscript EjecutableLotes.R")

0

In [82]:
# change directory to Resultado-ErroresCorregidos
os.chdir(EC)

# files in ErroresCorregidos folder
lista = os.listdir()

# move a file to Resultado-ErroresCorregidos
listadir = EC+"\\" + lista[0]
listadirnew = listadir.replace("ErroresCorregidos", "Resultado-ErroresCorregidos")

os.rename(listadir, listadirnew)



FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'C:\\Users\\fcolin\\Desktop\\ErroresCorregidos'

In [50]:
os.chdir(resultados)

if os.listdir(EC) == []:
    os.rmdir(EC)

Finalmente se eliminan todas las varibales que fueron creadas, se libera memoria y evita complicaciones en posteriores usos 

In [32]:
del  carpeta, files, paths, principales, imagen, new_label